Summary: need to go from all calls to calls from 8am-8pm and monday-friday

In [1]:
import csv
import collections
import pandas as pd
import numpy as np
import math
import json

In [2]:
clean = np.genfromtxt('..\Input_Data/cleaned_data.csv', delimiter=",", dtype=str) #210k x 17
a_i = np.genfromtxt('..\Input_Data/austin_incidents.csv', delimiter=",", dtype=str) #210k x 17

In [3]:
clean[0]

array(['', 'IncidentForeignKey', 'Radio_Name', 'Longitude_At_Assign_Time',
       'Latitude_At_Assign_Time', 'Time_Assigned', 'Time_Enroute',
       'Time_ArrivedAtScene', 'Time_Depart_Scene',
       'Time_Arrive_Destination', 'Time_Available', 'Time_Call_Cleared',
       'Call_Disposition', 'Longitude_Of_Emergency',
       'Latitude_Of_Emergency', 'transport_time', 'grid_time'],
      dtype='<U30')

In [6]:
clean[1]

array(['1', '23397873', 'M09', '-97.972866', '30.333385',
       '2019-01-01 19:43:36.000', '2019-01-01 19:45:27.030',
       '2019-01-01 19:48:39.770', '2019-01-01 20:19:00.870',
       '2019-01-01 20:24:43.093', '2019-01-01 20:51:12.877',
       '2019-01-01 20:51:12.877', 'Baylor Scott & White - Lakeway',
       '-97.961', '30.325', '192', '222.72'], dtype='<U30')

In [30]:
clean[3000:3100,-5]

array(['North Austin Hospital', 'Refusal', 'Dell Seton Med Ctr',
       'Saint Davids Med Ctr', 'False Alarm Call',
       'South Austin Hospital', 'Refusal', 'Call Cancelled',
       'South Austin Hospital', 'Refusal', 'No Patient',
       'Round Rock Hospital', 'Baylor Scott & White - Lakeway',
       'No Patient', 'Refusal', 'False Alarm Call', 'No Patient',
       'False Alarm Call', 'Refusal', 'False Alarm Call',
       'South Austin Hospital', 'No Patient', 'Refusal', 'Refusal',
       'No Patient', 'Call Cancelled', 'No Patient', 'No Patient',
       'False Alarm Call', 'Call Cancelled', 'False Alarm Call',
       'False Alarm Call', 'Round Rock Hospital', 'Refusal', 'No Patient',
       'False Alarm Call', 'Call Cancelled', 'Refusal', 'Refusal',
       'Refusal', 'No Patient', 'False Alarm Call', 'No Patient',
       'No Patient', 'False Alarm Call', 'No Patient', 'No Patient',
       'Call Cancelled', 'Refusal', 'False Alarm Call',
       'False Alarm Call', 'Seton Med Ctr', '

TypeError: list indices must be integers or slices, not tuple

In [4]:
import math
from datetime import *; from dateutil.relativedelta import *
import calendar
def csv2calendar(index,type_event):
    time1 = clean[index,type_event-1]  #note backend and clean data sets are off by one index
    #time1 = backend[index,type_event]
    if time1 == "":
        return -1
    [ymd,hmm] = time1.split()
    [y,m,d] = ymd.split('-')
    [h,minu,sec] = hmm.split(":")
    y = int(y)
    m = int(m)
    d = int(d)
    h = int(h)
    minu = int(minu)
    sec = math.floor(float(sec))
    timearrive = datetime(y, m, d, h, minu, sec)
    return timearrive

In [5]:
len(clean)

210108

In [35]:
nsamples = len(clean)
peak_hours = []
peak_hours.append(clean[0]) #headers

for i in range(1,nsamples):
    Time_Enroute = csv2calendar(i,7)
    #print(Time_Enroute)
    #peak hour
    if not ((Time_Enroute.hour < 21) and (Time_Enroute.hour > 7)):
        continue
        print(Time_Enroute.hour)
    #dow
    if not(Time_Enroute.weekday() < 5):
        continue
        print(Time_Enroute.weekday())
    #was it a call that was answered?
    #'No Patient' = 103k - 96835 = 7k
    #'Refusal' = 96835 - 76986 = 19849 (sheesh)
    #'Call Cancelled' 74197 = ~2k
    #'False Alarm Call' 74197 - 73851 = ~1k
    #'Dual w/ other ATCEMS Unit' 73851 - 66979 = 6872
    #'Referred MCOT (CHP Use Only)' 66830 = ~0
    #other 8k
    if (clean[i,-5] == 'No Patient') or (clean[i,-5] == 'Refusal') or (clean[i,-5] == 'Call Cancelled') or (clean[i,-5] == 'False Alarm Call') or (clean[i,-5] == 'Dual w/ other ATCEMS Unit') or (clean[i,-5] == 'Referred MCOT (CHP Use Only)') or  (clean[i,-5] == 'Other'):
        continue
    
    peak_hours.append(clean[i])

In [37]:
x = np.array(peak_hours)
x.shape

(60960, 17)

In [38]:
73851 - 66979

6872

In [41]:
np.savetxt('..\Output_Data/peakhours.csv', x, delimiter=',', fmt='%s') 

In [42]:
y = (np.genfromtxt("..\Output_Data/peakHours.csv", delimiter=",", dtype = str))
y.shape

(60960, 17)

In [44]:
y[1:200,-5]

array(['Baylor Scott & White - Lakeway', 'Saint Davids Med Ctr',
       'Saint Davids Med Ctr', 'Saint Davids Med Ctr',
       'Saint Davids Med Ctr', 'North Austin Hospital', 'Seton Northwest',
       'North Austin Hospital', 'South Austin Hospital',
       'Saint Davids Med Ctr', 'Round Rock Hospital', 'Seton Northwest',
       'Patient Contact', 'South Austin Hospital',
       'North Austin Hospital', 'North Austin Hospital',
       'Saint Davids Med Ctr', 'South Austin Hospital',
       'South Austin Hospital', 'South Austin Hospital',
       'North Austin Hospital', 'South Austin Hospital', 'Seton Med Ctr',
       'North Austin Hospital', 'Round Rock Hospital',
       'Saint Davids Med Ctr', 'North Austin Hospital',
       'South Austin Hospital', 'Round Rock Hospital', 'Seton Northwest',
       'Deceased on Scene', 'North Austin Hospital', 'Seton Med Ctr',
       'North Austin Hospital', 'Dell Seton Med Ctr', 'Heart Hospital',
       'North Austin Hospital', 'North Austin Hospita